<a href="https://colab.research.google.com/github/LucioFassarella/QCOP/blob/main/QCOP_minimizacao_por_inspecao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QISKIT: Carregamento

In [1]:
!pip install qiskit -U
!pip install qiskit_aer -U
!pip install qiskit-ibm-runtime -U

!pip install matplotlib
!pip install pylatexenc

import qiskit
qiskit.__version__

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.9/363.9 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currentl

'2.1.0'

In [2]:
# Qiskit: métodos básicos

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

from qiskit import transpile

from qiskit.visualization import plot_histogram, array_to_latex, plot_state_city

# Problemas de Otimização Combinatória: Hamiltoniano

Espaço das sequências binárias com $n \in \mathbb{N}$ termos:

$$
\mathcal{Z}(n) = \left\{(z_{0}, \dots, z_{n-1});\ z_0, \dots, z_{n-1} \in \left\{0,1\right\}\right\}.
$$

Função objetivo:

$$
C:S \rightarrow \mathbb{R},\ \ S \subseteq \mathcal{Z}(n).
$$

<!-- Para $z = (z_0,\dots, z_{n-1}) \in \mathcal{Z}(n)$ vale:

\begin{equation*}
	\begin{split}
		|z\rangle \langle z | &= |z_{n-1}\rangle \langle z_{n-1} |\otimes \dots \otimes |z_0\rangle \langle z_0 |\\
		&= \frac{I + (-1)^{z_{n-1}}Z_{n-1}}{2}\otimes \dots \otimes \frac{I + (-1)^{z_0}Z_{0}}{2}\\
		&= I + \sum_{k=1}^n\sum_{0 \le j_1 < \dots <j_k \le n-1}%
		(-1)^{z_{j_1} + \dots + z_{j_k}}Z_{j_1} \dots Z_{j_k}.
	\end{split}
\end{equation*}
-->

Hamiltoniano em termos dos operadores de Pauli é dada por:

\begin{equation}
	\begin{split}
		H_C &= \sum_{z \in S}C(z)|z\rangle \langle z |\\
		&= \frac{1}{2^n}\sum_{z \in S}C(z)%
		\left\lbrack I + \sum_{k=0}^{n-1}\sum_{0 \le j_0 < \dots <j_k \le n-1}%
		(-1)^{z_{j_1} + \dots + z_{j_k}}Z_{j_1} \dots Z_{j_k}
		\right\rbrack.
	\end{split}
\end{equation}

In [3]:
def funcao_Hamiltoniano(n = "int", C = "function", S = "None"):
    '''
    Função que constrói o Hamiltoniano em termos dos operadores de Pauli.

    Entrada:
        n: tipo = inteiro: --> número de termos.
        C: tipo = função: --> função objetivo.
        S: tipo = lista: --> subconjunto das sequências binárias com n termos.

    Saída:
        H_C: tipo = lista --> Hamiltoniano em termos dos operadores de Pauli.

    Métodos:
        inspect.isfunction(): < https://docs.python.org/3/library/inspect.html >
        copy(): < https://docs.python.org/3/library/copy.html >
        itertools.combinations(): < https://docs.python.org/3/library/itertools.html >
        SparsePauliOp: < https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.SparsePauliOp >

    Funções:
        sequencias_binarias(n = "int"): --> lista de todas as sequências binárias com n termos.
    '''

    # Métodos

    from qiskit.quantum_info import SparsePauliOp
    from copy import deepcopy as dcopy
    import inspect

    # Condições

    if type(n) != int or n <= 0:
        print("ERRO: A entrada deve ser um inteiro positivo.")
        return []

    if inspect.isfunction(C) == False:
        print("ERRO: A entrada deve ser uma função definida em sequências binárias.")
        return []

    if S != "None":
        if type(S) != list:
            print("ERRO: A entrada deve ser nada ou uma lista de sequências binárias.")
            return []

    # Construção de HC

    if S == "None":
        sequencias = funcao_SequenciasBinarias(n)
    else:
        sequencias = dcopy(S)

    for sequencia in sequencias:
        sequencia.reverse()

    indices = funcao_Subsequencias([k for k in range(n)])

    operadores = []

    for sequencia in sequencias:
        coeficiente = C(sequencia)/2**n

        if coeficiente == 0:
            pass

        else:
            operadores.append(("I", [0] , coeficiente))

            for indice in indices:
                sinal = (-1)**sum(sequencia[j] for j in indice)
                operadores.append((len(sequencia)*"Z", indice , sinal*coeficiente))

    H_C = SparsePauliOp.from_sparse_list(operadores, num_qubits=n)

    return H_C

In [4]:
def funcao_SequenciasBinarias(n = "int"):
    '''
    Função que contrói o conjunto das sequências binárias com n termos.

    Entrada:
        n: tipo = inteiro: --> número de termos.

    Saída:
        Z_n: tipo = lista --> sequências binárias com n termos.

    Métodos:
        copy(): < https://docs.python.org/3/library/copy.html >
    '''
    # Métodos

    from copy import deepcopy as dcopy

    # Condição

    if type(n) != int or n <= 0:
        print("ERRO: A entrada deve ser um inteiro positivo.")
        return []

    # Parte principal

    if n == 1:
        Z_n = [[0], [1]]

    else:
        sequencias = funcao_SequenciasBinarias(n - 1)
        sequencias_0 = dcopy(sequencias)
        sequencias_1 = dcopy(sequencias)
        for sequencia in sequencias_0:
                sequencia.insert(0,0)
        for sequencia in sequencias_1:
                sequencia.insert(0,1)
        Z_n = sequencias_0 + sequencias_1
    return Z_n

In [5]:
def funcao_Subsequencias(sequencia):
    '''
    Função que constrói todas as subsequências de uma dada sequência.

    Entrada:
        sequencia: tipo = list --> sequência

    Saída:
        lista_subsequencias = list --> lista de todas as subsequências da sequência.

    Métodos:
        intertools.combinations(): < https://docs.python.org/3/library/itertools.html >

    '''
    # Método

    from itertools import combinations

    # Condições:

    if type(sequencia) != list:
        print("ERRO: A entrada deve ser uma lista.")
        return []

    # Loop:

    sublistas = []
    for r in range(len(sequencia) + 1):
        r_sublistas = [list(combo) for combo in combinations(sequencia, r)]
        sublistas.extend(r_sublistas)
    return sublistas[1:]

In [6]:
def funcao_EstadoBase(sequencia_binaria = list):
    '''
    Função que constrói o circuito que prepara o estado da base computacional com os bits dados.

    Entrada:
        sequencia_binaria: tipo = list --> lista de bits.

    Saída:
        circuito: tipo = QuantumCircuit --> circuito que prepara o estado da base computacional.

    Métodos:
        qiskit.circuit.QuantumCircuit(): < https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html >

    '''

    # Métodos

    from qiskit import QuantumCircuit

    # Condições:

    if type(sequencia_binaria) != list:
        print("ERRO: A entrada deve ser uma lista de bits.")
        return []

    # Principal

    n = len(sequencia_binaria)

    circuito = QuantumCircuit(n)

    for i in range(n):
        if sequencia_binaria[i] == 1:
            circuito.x(i)

    return circuito

In [7]:
def funcao_Base(n = "int"):
    '''
    Função que constrói a lista dos circuitos que preparam os estados da base computacional de n qubits.

    Entrada:
        n: tipo = inteiro: --> número de qubits.

    Saída:
        lista_circuitos: tipo = list --> lista de circuitos que preparam estados da base computacional

    Métodos:
        qiskit.circuit.QuantumCircuit(): < https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html >

    Funções:
        sequencias_binarias(n = "int"): --> lista de todas as sequências binárias com n termos.
    '''

    # Métodos

    from qiskit import QuantumCircuit

    # Condição

    if type(n) != int or n <= 0:
        print("ERRO: A entrada deve ser um inteiro positivo.")
        return []

    # Principal

    indices = funcao_SequenciasBinarias(n)

    lista_circuitos = []

    for indice in indices:
        qc = QuantumCircuit(n)
        for i in range(len(indice)):
            if indice[i] == 1:
                qc.x(i)
        lista_circuitos.append(qc)

    return lista_circuitos

In [8]:
def funcao_ValorEsperado(qc_estado, hamiltoniano, backend):
    '''
    Função que fornece o valor esperado de um operador (SparsePauliOp) em um estado (QuantumCircuit)
    utilizando o EstimatorV2 do Qiskit Runtime para um backend específico.

    Entradas:
        estado: qiskit.QuantumCircuit()
        hamiltoniano: qiskit.quantum_info.SparsePauliOp()
        backend: qiskit.providers.backend.Backend()

    Saída:
        valor_esperado: float

    Métodos:
        qiskit_ibm_runtime.EstimatorV2():
             < https://qiskit.org/ecosystem/ibm-runtime/stubs/qiskit_ibm_runtime.EstimatorV2.html >
        qiskit.transpiler.preset_passmanagers.generate_preset_pass_manager():
             < https://quantum.cloud.ibm.com/docs/en/api/qiskit/transpiler_preset >

    '''

    # Métodos

    from qiskit_ibm_runtime import EstimatorV2 as Estimator
    from qiskit.quantum_info import SparsePauliOp
    from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

    # Condições:

    if type(qc_estado) != QuantumCircuit:
        print("ERRO: A entrada deve ser um circuito quântico que prepara um estado")
        return None

    if type(hamiltoniano) != SparsePauliOp:
        print("ERRO: A entrada deve ser um operador do tipo SparsePauliOp.")
        return None

    if backend == None:
        print("ERRO: A entrada deve ser um backend do qiskit.ibm.runtime.")
        return None

    # Estimar

    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

    isa_estado = pm.run(qc_estado)
    isa_hamiltoniano = hamiltoniano.apply_layout(isa_estado.layout)

    estimator = Estimator(mode=backend)

    job = estimator.run([(isa_estado, isa_hamiltoniano)]) # calcula [ < estado|hamiltoniano|estado > ]

    pub_result = job.result()[0]

    return pub_result.data.evs

In [10]:
def MinQCOP(n = "int", C = "function", backend = None):

    '''

    Função que constrói o Hamiltoniano em termos dos operadores de Pauli.

    Entrada:
        n: int --> número de termos.
        C: function --> função objetivo.
        S: list --> subconjunto das sequências binárias com n termos.
        backend: qiskit.providers.backend.Backend()

    Saída:
        indice: list --> bits do estado da base computacional que minimiza o hamiltoniano de custo.
        valor_esperado: float --> mínimo valor esperado do hamiltoniano de custo.

    Métodos:
        inspect.isfunction(): < https://docs.python.org/3/library/inspect.html >
        copy(): < https://docs.python.org/3/library/copy.html >
        itertools.combinations(): < https://docs.python.org/3/library/itertools.html >
        SparsePauliOp: < https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.quantum_info.SparsePauliOp >

    Funções:
        sequencias_binarias(n = "int"): --> lista de todas as sequências binárias com n termos.

    '''

    # if S == "None":
    #     sequencias = funcao_SequenciasBinarias(n)
    # else:
    #     sequencias = dcopy(S)

    sequencias_binarias = funcao_SequenciasBinarias(n)

    hamiltoniano = funcao_Hamiltoniano(n, C, sequencias_binarias)

    if backend is None:
        from qiskit_ibm_runtime.fake_provider import FakeWashingtonV2
        backend = FakeWashingtonV2()


    sequencias_minimais = [sequencias_binarias[0]]
    estado = funcao_EstadoBase(sequencias_minimais[0])
    valor_esperado_minimal = funcao_ValorEsperado(estado, hamiltoniano, backend)

    for seq in sequencias_binarias:
        estado = funcao_EstadoBase(seq)
        valor_esperado = funcao_ValorEsperado(estado, hamiltoniano, backend)
        if valor_esperado == valor_esperado_minimal:
            sequencias_minimais.append(seq)
        if valor_esperado < valor_esperado_minimal:
            sequencias_minimais = [seq]
            valor_esperado_minimal = valor_esperado

    return sequencias_minimais, valor_esperado_minimal

# Testes

In [11]:
# Teste da função SequenciasBinarias():

sequencias_exemplo = funcao_SequenciasBinarias(3)
print(f"Sequências binárias de 3 termos: {sequencias_exemplo}")

Sequências binárias de 3 termos: [[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]]


In [12]:
# Teste da função Subsequencias():

sequencia_exemplo = [1, 2, 3]
sub_sequencias = funcao_Subsequencias(sequencia_exemplo)
print(f"Sequência: {sequencia_exemplo}")
print(f"Subsequências: {sub_sequencias}")

Sequência: [1, 2, 3]
Subsequências: [[1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]


In [13]:
# Teste da função Hamiltoniano():

def C(x):
    return sum(x)

S = funcao_SequenciasBinarias(3)
H = funcao_Hamiltoniano(3, C)
print(H)

SparsePauliOp(['III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ', 'III', 'IIZ', 'IZI', 'ZII', 'IZZ', 'ZIZ', 'ZZI', 'ZZZ'],
              coeffs=[ 0.125+0.j, -0.125+0.j,  0.125+0.j,  0.125+0.j, -0.125+0.j, -0.125+0.j,
  0.125+0.j, -0.125+0.j,  0.125+0.j,  0.125+0.j, -0.125+0.j,  0.125+0.j,
 -0.125+0.j,  0.125+0.j, -0.125+0.j, -0.125+0.j,  0.25 +0.j, -0.25 +0.j,
 -0.25 +0.j,  0.25 +0.j,  0.25 +0.j, -0.25 +0.j, -0.25 +0.j,  0.25 +0.j,
  0.125+0.j,  0.125+0.j,  0.125+0.j, -0.125+0.j,  0.125+0.j, -0.125+0.j,
 -0.125+0.j, -0.125+0.j,  0.25 +0.j, -0.25 +0.j,  0.25 +0.j, -0.25 +0.j,
 -0.25 +0.j,  0.25 +0.j, -0.25 +0.j,  0.25 +0.j,  0.25 +0.j,  0.25 +0.j,
 -0.25 +0.j, -0.25 +0.j, -0.25 +0.j, -0.25 +0.j,  0.25 +0.j,

In [14]:
# Teste da função Base()

n = 3
estados = funcao_Base(n)
print(f"Número de qubits = {n}. Número de circuitos: 2**{n} = {len(estados)}")
for estado in estados:
    display(estado.draw())

Número de qubits = 3. Número de circuitos: 2**3 = 8


q_0: 
     
q_1: 
     
q_2:

q_0: ─────
          
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘

q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘
q_2: ─────

q_0: ─────
     ┌───┐
q_1: ┤ X ├
     ├───┤
q_2: ┤ X ├
     └───┘

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
          
q_2: ─────

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     └───┘
q_2: ─────

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     ├───┤
q_2: ┤ X ├
     └───┘

In [15]:
# Teste da função EstadoBase()

sequencia_exemplo = [1, 0, 1]
estado = funcao_EstadoBase(sequencia_exemplo)
display(estado.draw())

┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘

In [16]:
# Teste da função ValorEsperado()

from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel="ibm_cloud",
    token="7zAryvcpz2cTOgAbliiOL_A4qMyxg5Vq3nuHiBs1VENt",
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/8e99d0dd96d44fd08e92d06df59c1d0a:a600aec9-e7f1-40c6-95ac-ae168bf213b5::"
    )

## Save account to disk and save it as the default.
#QiskitRuntimeService.save_account(channel="ibm_cloud", token="<IBM Cloud API key>", instance="<IBM Cloud CRN>", name="account-name", set_as_default=True)

## Load the saved credentials
#service = QiskitRuntimeService(name="account-name")

estado = QuantumCircuit(3)
estado.x(0)
estado.x(1)
estado.x(2)

def C(x):
    return 2*(x[1]-1)**2 + sum(x)

hamiltoniano = funcao_Hamiltoniano(3, C)


from qiskit_ibm_runtime.fake_provider import FakeWashingtonV2
backend = FakeWashingtonV2()

valor_esperado = funcao_ValorEsperado(estado, hamiltoniano, backend)
print(valor_esperado)

2.977783203125


In [17]:
# Teste da função MinQCOP():

from qiskit_ibm_runtime.fake_provider import FakeWashingtonV2
backend = FakeWashingtonV2()

minimix = MinQCOP(3, C, backend = backend)
display(minimix)

print(f"Sequências minimais = {minimix[0]}")
print(f"Valor esperado minimal = {minimix[1]}")

([[0, 1, 0]], array(1.03149414))

Sequências minimais = [[0, 1, 0]]
Valor esperado minimal = 1.031494140625
